In [33]:

import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
import numpy as np
from fake_useragent import UserAgent
from urllib.request import Request, urlopen
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.data import url2pathname

from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

from bs4 import BeautifulSoup

from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english')) 
stop_words.update(['le', 'eu', 'span', 'ago', 'pp', 'ue', 'div', 'src', 'page', 'egp', 'url', 'cdn', 'alt', 'com', 'net', 'org', 'cdn', 'img', 'google','eg','usd','http','https','net','us','eg'])




import mysql.connector

# Connect to the database
mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="seopro"
)

# Create a cursor object
mycursor = mysql.cursor()

# Execute an SQL query to select the last row from the table
mycursor.execute("SELECT uid, niche FROM given_niche ORDER BY id DESC LIMIT 1")

# Fetch the last row from the table
niche_arr = mycursor.fetchone()

# # Convert niche[0] to a string
# niche= str(niche_arr[0])

# # Print the last row as a string
# print(niche)

# Extract the id and niche values from the row
uid = niche_arr[0]
niche = niche_arr[1]

# Print the niche and id values
print(niche)
print(uid)


def get_text(url):
    try:
        req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req,timeout=5).read()
        soup = BeautifulSoup(webpage, "html.parser")
        texts = soup.findAll(text=True)
        res=u" ".join(t.strip() for t in texts if t.parent.name not in ['style', 'script', 'head', 'title', 'meta', '[document]'])
        return(res)
    except:
        return False
    
"""Step 2: Get the URLs from competitors

"""

def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)

        return response
    except requests.exceptions.RequestException as e:
        print(e)
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    links = list(response.html.absolute_links)
    # google_domains = ('https://www.google.', 
    #                   'https://google.', 
    #                   'https://webcache.googleusercontent.', 
    #                   'http://webcache.googleusercontent.', 
    #                   'https://policies.google.',
    #                   'https://support.google.',
    #                   'https://maps.google.')
    
    google_domains = ('https://www.google.', 
                      'https://google.')
     

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

# niche = input("enter your niche")
# scrape_google(niche)

"""Step 3: Analyse the text and get the most important words."""

# niche = input("enter your niche: ")
links=scrape_google(niche)

text=[]
for i in links:
  t=get_text(i)
  if t:
    text.append(t)

    word_tokens = word_tokenize(t)

   
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    


p_stemmer = PorterStemmer()




for word in filtered_sentence:
    # if len(word) >= 3:
    word+' --> '+p_stemmer.stem(word)

from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')



for word in filtered_sentence:
    if len(word) >=4:
        word+' --> '+s_stemmer.stem(word)
u=p_stemmer.stem(word)
import mysql.connector

########
# Modify the tf_idf_analysis function to return a DataFrame
def tf_idf_analysis(keyword):
    v = TfidfVectorizer(min_df=1,analyzer='word',ngram_range=(1,2),stop_words=list(stop_words))
    x = v.fit_transform(text)
    f = pd.DataFrame(x.toarray(), columns = v.get_feature_names_out())
    d=pd.concat([pd.DataFrame(f.mean(axis=0)),pd.DataFrame(f.max(axis=0))],axis=1)
    tf=pd.DataFrame((f>0).sum(axis=0))
    d=d.reset_index().merge(tf.reset_index(),on='index',how='left')
    d.columns=['word','average_tfidf','max_tfidf','frequency']
    d['frequency']=round((d['frequency']/len(text))*100)
    d['max_tfidf'] = d['max_tfidf'].round(3)
    d['average_tfidf'] = d['average_tfidf'].round(3)

    d= d[d['word'].str.isalpha()].sort_values('frequency',ascending=False).head(35)
    
    return d


# Call the tf_idf_analysis function and store the output in a DataFrame variable
output_df = tf_idf_analysis(u)
print (output_df)
# Convert the DataFrame to a dictionary
output_dict = output_df.to_dict('records')

# Connect to the database
mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="seopro"
)

# Insert the data into the database
mycursor = mysql.cursor()
sql = "INSERT INTO keywords (word, average_tfidf, max_tfidf, frequency, niche, uid) VALUES (%s, %s, %s, %s, %s, %s)"
for row in output_dict:
    val = (row['word'], row['average_tfidf'], row['max_tfidf'], row['frequency'], niche, uid)
    mycursor.execute(sql, val)
mysql.commit()

# Print a message to confirm that the data has been saved
print("Data has been saved to the database.")






[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


pllot
5


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12020\2751849280.py:69: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = soup.findAll(text=True)


              word  average_tfidf  max_tfidf  frequency
8729       privacy          0.004      0.015       70.0
9584        search          0.010      0.041       65.0
11623          use          0.012      0.060       61.0
1617          best          0.006      0.028       61.0
7347           new          0.052      0.261       57.0
2474         class          0.018      0.115       57.0
2395        choose          0.011      0.057       57.0
5252          help          0.006      0.020       52.0
7599           one          0.021      0.118       52.0
4921           get          0.012      0.091       52.0
9646           see          0.023      0.146       52.0
2786       contact          0.005      0.020       52.0
8104         pilot          0.107      0.381       52.0
8537        policy          0.004      0.018       52.0
5844   information          0.006      0.027       52.0
4665        follow          0.004      0.011       48.0
3824           end          0.012      0.170    